In [11]:
import pandas as pd

import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

from datasets import Dataset

import json

In [12]:
# Medication file
kisim_medication = pd.read_csv(paths.DATA_PATH_SEANTIS/"kisim_medication.csv")

# Drop nan values
kisim_medication = kisim_medication.dropna(subset=["medication_name"])

# Select columns
kisim_medication = kisim_medication[["medication_name", "research_id"]].rename(columns={"medication_name": "text", "research_id": "rid"})

# Remove empty strings
kisim_medication = kisim_medication[kisim_medication["text"] != ""]

# Select random 100 rows
kisim_medication_sample = kisim_medication.sample(100, random_state=42)

for index, row in kisim_medication_sample.iterrows():
    print(row["text"])
    print(50*"-")

Prednison 100 mg	1-0-0		24.09. - 07.10.2016
Prednison 80 mg		1-0-0		08.10. - 14.10.2016
Prednison 60 mg		1-0-0		15.10. - 21.10.2016
Prednison 40 mg		1-0-0		22.10.2016 bis auf weiteres

Pantozol 40 mg		1-0-0 		für die Dauer der Prednison-Behandlung
--------------------------------------------------
Auge rechts:
Floxal AT 4x/d für 5 Tage
Vitamine A AS zur Nacht
--------------------------------------------------
Volare Handgelenksschiene zur Nacht, bitte 1x für beide Hände

Dg.: CTS bds
--------------------------------------------------
Ebrufen 200 mg
--------------------------------------------------
Paracetamol 500 Hänseler neue Formel Tabl 20 (teilbar)
bei Bedarf


Ibuprofen Adico Filmtabl 400 mg 50 
bei Bedarf

Dauerrezept
--------------------------------------------------
Nexium MUPS-Tabl 40 mg 14 Stück Einnahme: 1-0-0
Zolpidem Winthrop Filmtabl 10mg 10 Stück (teilbar) Einnahme: 0-0-1
--------------------------------------------------
Einlagen
----------------------------------------

Some entries don't have medications in them but stuff like: "Bauchgurt" or "Blutdruckmessgerät".
Other entries don't have a full medication intake, like "Sirdalud MR (Ret Kaps 6 mg)"

In [13]:
med_dict ={
    "avonex": "Avonex",
    "betaferon": "Betaferon",
    "plegridy": "Plegridy",
    "copaxone": "Copaxone",
    "glatiramyl": "Glatiramyl",
    "aubagio": "Aubagio",
    "tecfidera": "Tecfidera",
    "gilenya": "Gilenya",
    "tysabri": "Tysabri",
    "ocrevus": "Ocrevus",
    "lemtrada": "Lemtrada",
    "novantron": "Novantron",
    "endoxan": "Endoxan",
    "mabthera": "MabThera",
    "imurek": "Imurek",
    "mayzent": "Mayzent",
    "medrol": "Medrol",
    "solu_medrol": "Solu-Medrol",
    "solumedrol": "Solumedrol",
    "cortison": "Cortison",
    "interferon_beta_1a": "Interferon beta-1a",
    "interferon_beta_1b": "Interferon beta-1b",
    "peginterferon_beta_1a": "Peginterferon beta-1a",
    "glatirameracetat": "Glatirameracetat",
    "teriflunomid": "Teriflunomid",
    "dimethylfumarat": "Dimethylfumarat",
    "fingolimod": "Fingolimod",
    "natalizumab": "Natalizumab",
    "ocrelizumab": "Ocrelizumab",
    "alemtuzumab": "Alemtuzumab",
    "mitoxantron": "Mitoxantron",
    "cyclophosphamid": "Cyclophosphamid",
    "rituximab": "Rituximab",
    "azathioprin": "Azathioprin",
    "siponimod": "Siponimod",
    "glucocorticosteroid": "Glucocorticosteroid",
    "kalium": "Kalium",
    "unknown": "unknown"
}

In [14]:
# Check if medication from sample text is in the dictionary
matches = 0
for index, row in kisim_medication_sample.iterrows():
    for value in med_dict.values():
        if value.lower() in row["text"].lower():
            print(row["text"])
            print(med_dict[value.lower()])
            print(50*"-")
            matches += 1

Frau Fleseriu-Erhan Adriana geb. 29.8.1972
Adresse: Fliederweg 3, 5244 Birrhard
Versicherung: Sanitas Nr.: 99396702

Gültigkeit: 1 Jahr

Tecfidera 240mg 1-0-1
Tecfidera
--------------------------------------------------
Prednison-Ausschleichschema:

Prednison 15 mg für 5 Tage
folgend: Prednison 12,5 mg für 5 Tage
		Prednison 10 mg für 5 Tage
		Prednison 7,5 mg für 5 Tage
		Prednison 5 mg für 5 Tage
		Prednison 2,5 mg für 5 Tage

Esomeprazol 40 mg für die Dauer der Cortisoneinnahme
Cortison
--------------------------------------------------
für 1 Jahr 

Avanex:  Vers.-Nr. 96472969 

Tecfidera 120mg 1-0-1 	für 1 Woche
ab 2. Woche Tecfidra 240mg 1-0-1 

Tecfidera
--------------------------------------------------
Medrol 100 mg 10-0-0 po 4 Tage
Pantoprazol 40 mg 1-0-0- 7 Tage
Medrol
--------------------------------------------------
Gilenya 0,5 mg p.o.	1-0-0

Assura Kranken- und Unfallversicherung
Vers.-Nr. 00000914176

1 Jahr gültig
Gilenya
------------------------------------------------

In [15]:
# Create Dataset
kisim_sample = Dataset.from_dict({
    "text": kisim_medication_sample["text"],
    "rid": kisim_medication_sample["rid"],
})

In [16]:
# Save datasets
os.makedirs(paths.DATA_PATH_PREPROCESSED/"medication", exist_ok=True)
kisim_medication_sample.to_csv(paths.DATA_PATH_PREPROCESSED/"medication/kisim_medication_sample.csv", index=False)
kisim_sample.save_to_disk(paths.DATA_PATH_PREPROCESSED/"medication/kisim_medication_sample")

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

## Prompting with Outlines

The following part is used to create the files for the prompting task with outlines. First we create a list of known MS medications (existed in the old project). Then we write a suitable prompt.


In [17]:
# Look at different medication formats:
for idx, text in enumerate(kisim_sample["text"]):
    print(5*"---")
    print(idx)
    print(text)

---------------
0
Prednison 100 mg	1-0-0		24.09. - 07.10.2016
Prednison 80 mg		1-0-0		08.10. - 14.10.2016
Prednison 60 mg		1-0-0		15.10. - 21.10.2016
Prednison 40 mg		1-0-0		22.10.2016 bis auf weiteres

Pantozol 40 mg		1-0-0 		für die Dauer der Prednison-Behandlung
---------------
1
Auge rechts:
Floxal AT 4x/d für 5 Tage
Vitamine A AS zur Nacht
---------------
2
Volare Handgelenksschiene zur Nacht, bitte 1x für beide Hände

Dg.: CTS bds
---------------
3
Ebrufen 200 mg
---------------
4
Paracetamol 500 Hänseler neue Formel Tabl 20 (teilbar)
bei Bedarf


Ibuprofen Adico Filmtabl 400 mg 50 
bei Bedarf

Dauerrezept
---------------
5
Nexium MUPS-Tabl 40 mg 14 Stück Einnahme: 1-0-0
Zolpidem Winthrop Filmtabl 10mg 10 Stück (teilbar) Einnahme: 0-0-1
---------------
6
Einlagen
---------------
7
Amoxicillin Sandoz (Disp Tabl 1000 mg)
---------------
8
Fampyra 10mg 1-0-1 per os
---------------
9
Burgerstein Vitamin D3 (Kaps)
---------------
10
Novalgin (Filmtabl 500 mg)
---------------
11
Cetall

In [18]:
# Prompt
task_instruction ="""Your task is to extract specific information from medication descriptions. 
The input for this task is a list of medication descriptions, a report or doctors recipe, and the output should be a complete list of dictionaries (one per medication) with the following keys:
- name (str): The name of the medication.
- dose (float): The dose of the medication.
- unit (str): The unit of the dose (mg, ug, g, stk).
- morning (float): The dose to be taken in the morning.
- noon (float): The dose to be taken at noon.
- evening (float): The dose to be taken in the evening.
- night (float): The dose to be taken at night.

The output format should look like this:
[
    {name: "MedicationName1", dose: MedicationDose1, unit: "MedicationDoseUnit1", morning: MorningDose1, noon: NoonDose1, evening: EveningDose1, night: NightDose1},
    {name: "MedicationName2", dose: MedicationDose2, unit: "MedicationDoseUnit2", morning: MorningDose2, noon: NoonDose2, evening: EveningDose2, night: NightDose2},
    ...
]

- The MedicationName can consist of multiple words with whitespace and should be returned as a single string. If you don't find any mentions of medications or drugs, it should be represented as "unknown".
- The MedicationDose should be in one of the following formats: mg, ug, g, stk, ml, tropfen, unknown. If the MedicationDose is not detected, it should be represented as "unknown".
- The MedicationDoseUnit is a float value. If the MedicationDoseUnit is not detected, it should be represented as -99.
- The intake doses over the day can be given in several ways:
    - If the amount of doses is given in the form of float-float-float, it corresponds to MorningDose-NoonDose-EveningDose with NightDose being 0.
    - If the amount of doses is given in the form float-float-float-float, it corresponds to MorningDose-NoonDose-EveningDose-NightDose.
    - If the amount of doses is given in the form of a set number of intakes per day, you can put all doses for the MorningDose.
    - If an intake schema like the ones above is not detected, all doses should be represented as -99.
"""

system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don’t know the answer to a question, please don’t share false information.
"""

with open(paths.DATA_PATH_PREPROCESSED/"medication/task_instruction.txt", "w") as file:
    file.write(task_instruction)

with open(paths.DATA_PATH_PREPROCESSED/"medication/system_prompt.txt", "w") as file:
    file.write(system_prompt)

In [19]:
# Examples for few shot based on the text formats I have seen
examples = [{"text": "1 OP Cipralex 20 mg 0.5-0-0 für 4 Tag\n 3 OP Propranolol 35g 3-0-1-0", "labels" : '[{name: "Cipralex", unit: "mg", amount: 20, morning: 0.5, noon: 0, evening: 0, night: 0}, {name: "Propranolol", unit: "g", amount: 35, morning: 3, noon: 0, evening: 1, night: 1}]'},
           {"text": "Blutdruckmessgerät über einen Zeitraum von 3 Wochen", "labels": '[{name: "Blutdruckmessgerät", unit: "unknown", amount: 1, morning: -99, noon: -99, evening: -99, night: -99}]'},
           {"text": "Rezept für Propranolol, einmal täglich, 0.5mg", "labels": '[{name: "Propranolol", unit: "mg", amount: 0.5, morning: 1, noon: 0, evening: 0, night: 0}]'}]

# Save dictionary
with open(paths.DATA_PATH_PREPROCESSED/"medication/examples.json", "w") as file:
    json.dump(examples, file, indent=4)